**Authors:** Jozef Hanč, Martina Hančová  <br> *[Faculty of Science](https://www.upjs.sk/en/faculty-of-science/?prefferedLang=EN), P. J. Šafárik University in Košice, Slovakia* <br> emails: [martina.hancova@upjs.sk](mailto:martina.hancova@upjs.sk)
***

# <font color = brown, size=6> Cross-checking $\mathcal{GDD}$ pdf values (Arb)</font>

<font size=4> Computational tools - arbitrary-precision C libary: </font>  **<font size=4>Arb</font>**  

**Arb in Sage**
https://doc.sagemath.org/html/en/reference/rings_numerical/sage/rings/complex_arb.html

In [1]:
# python libraries
import numpy as np
from numpy import inf as INF, array as v
import platform as pt
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')

from time import time
import math, cmath
import scipy
scipy.__version__, np.__version__

('1.5.2', '1.19.1')

## Python procedures and functions

In [2]:
# accuracy commands
def abs_errs(df1, df2):
    N = len(df1)
    errors = [abs(df1[i]-df2[i]) for i in range(N)]
    return errors

def accuracy(df1, df2):
    return max(abs_errs(df1,df2))

In [3]:
# approximate formulas for precisions expressed in bits and decimal places
bits = lambda d:round((d+1)*ln(10)/ln(2)) 
dps = lambda b:round(b*ln(2)/ln(10) - 1)

In [4]:
# 668 bits precision in decimal places
dps(668)

200

In [5]:
# Arb numbers and functions
prec = 668
Arb = ComplexBallField(prec)

R_Arb = lambda z: Arb(z,0)
U_Arb = lambda a,b,z: Arb(z,0).hypergeometric_U(a,b)
e_Arb = lambda z: Arb(z,0).exp()
G_Arb = lambda z: Arb(z,0).gamma()
erf_Arb = lambda z: Arb(z,0).erf()

In [7]:
RRf = RealField(prec)

## Defining $f(z)$

$
f(z)=
\dfrac{\beta_{1}^{\alpha_{1}} \beta_{2}^{\alpha_{2}}}{\beta^{\alpha-1}}
\begin{cases}
{\dfrac{e^{z \beta_{2}}}{\Gamma\left(\alpha_{2}\right)} U\left(1-\alpha_{2}, 2-\alpha,-z \beta\right),} & {z<0} \\[12pt]
\begin{array}{cc} 
\frac{\Gamma(\alpha-1)}{\beta^{a / 2-1} \Gamma\left(\alpha_{1}\right) \Gamma\left(\alpha_{2}\right)}, & \scriptstyle 1<\alpha \\ 
\infty,  & \scriptstyle 0<\alpha \leq 1 
\end{array}, & z=0 \\[12pt] 
{\dfrac{e^{-z \beta_{1}}}{\Gamma\left(\alpha_{1}\right)} U\left(1-\alpha_{1}, 2-\alpha, z \beta\right),} & {z>0}
\end{cases}
$


$\alpha=\alpha_{1}+\alpha_{2}, \quad \beta=\beta_{1}+\beta_{2}$

In [8]:
# parameters
a1, b1 = R_Arb(1/2), R_Arb(1)
a2, b2 = R_Arb(17/2), R_Arb(93)

In [9]:
a, b = a1+a2, b1+b2
c = b1^a1*b2^a2/b^(a-1)
cp, cm = c/G_Arb(a1), c/G_Arb(a2)

In [10]:
# defining f(z)
fm = lambda z: cm*e_Arb( R_Arb(z)*b2)*U_Arb(1-a2,2-a,-b*R_Arb(z))
f0 = c*G_Arb(a-1)/(G_Arb(a1)*G_Arb(a2))
fp = lambda z: cp*e_Arb(-R_Arb(z)*b1)*U_Arb(1-a1,2-a, b*R_Arb(z))

f  = lambda z: fp(z) if z > 0 else fm(z) if z < 0 else f0

In [11]:
f(2)

[0.0482250429701290776115038844756988061333201564362886719369287862297731253154395847360022755351782246659 +/- 6.78e-104]

In [12]:
f(2).diameter()

1.4139811e-103

In [13]:
f(2).mid()

0.048225042970129077611503884475698806133320156436288671936928786229773125315439584736002275535178224665867607537204775353015360870114578360692376004555018200333880096312352968300441534558594470212189881

# Cross-checking PARI values

In [14]:
# pdf quadruple precision values
N = 5
dparipdf = {str(10**(n+1)):np.loadtxt('Pari_Sage_pdf'+str(10**(n+1))+'.txt', delimiter=',', 
                                      dtype=np.longdouble) for n in range(N)}

In [15]:
dparipdf['10']

array([1.03379229e-106, 2.83772520e-076, 2.93861976e-046, 2.39164468e-017,
       1.03565906e+000, 2.14093530e-001, 7.34128746e-002, 2.80789597e-002,
       1.12843517e-002, 4.66541759e-003], dtype=float128)

In [16]:
darb = dict()
for n in [1..N]:
    tic = time()
    
    points = [-3+7/(10^n-1)*(i-1) for i in [1..10^n]]
    darb[str(10^n)] = [f(val) for val in points]
    
    toc = time()-tic; 
    print('10^'+str(n),' runtime =',toc,'s')

10^1  runtime = 0.025335073471069336 s
10^2  runtime = 0.22606682777404785 s
10^3  runtime = 2.196476697921753 s
10^4  runtime = 22.11991548538208 s
10^5  runtime = 219.86930203437805 s


In [17]:
darbpdf = {str(10^n): [item.mid() for item in darb[str(10^n)]] for n in [1..N]}
darbpdferrs = {str(10^n): max([item.diameter() for item in darb[str(10^n)]]) for n in [1..N]}

In [18]:
# diameters
darbpdferrs

{'10': 2.0358705e-20,
 '100': 2.0358705e-20,
 '1000': 1.7151512e-19,
 '10000': 3.1987354e-19,
 '100000': 3.3480718e-19}

In [19]:
Errors = {str(10^n): (accuracy(darbpdf[str(10^n)], dparipdf[str(10^n)])) for n in [1..N]}

In [20]:
Errors

{'10': 0.000000000000000,
 '100': 0.000000000000000,
 '1000': 0.000000000000000,
 '10000': 2.77555756156289e-17,
 '100000': 5.55111512312578e-17}

***
<a id=references></a>
# <font color=brown> References </font>
This notebook belongs to supplementary materials of the paper submitted to Journal of Statistical Computation and
Simulation and available at  <https://arxiv.org/abs/2105.04427>.
* Hančová, M., Gajdoš, A., Hanč, J. (2021). A practical, effective calculation of gamma difference distributions with open data science tools. arXiv:2105.04427 [cs, math, stat], https://arxiv.org/abs/2105.04427

### Abstract of the paper

At present, there is still no officially accepted and extensively verified implementation of computing the gamma difference distribution allowing unequal shape parameters. We explore four computational ways of the gamma difference distribution with the different shape parameters resulting from time series kriging, a forecasting approach based on the best linear unbiased prediction, and linear mixed models. The results of our numerical study, with emphasis on using open data science tools, demonstrate that our open tool implemented in high-performance Python(with Numba) is exponentially fast, highly accurate, and very reliable. It combines numerical inversion of the characteristic function and the trapezoidal rule with the double exponential oscillatory transformation (DE quadrature). At the double 53-bit precision, our tool outperformed the speed of the analytical computation based on Tricomi's $U(a, b, z)$ function in CAS software (commercial Mathematica, open SageMath) by 1.5-2 orders. At the precision of scientific numerical computational tools, it exceeded open SciPy, NumPy, and commercial MATLAB 5-10 times. The potential future application of our tool for a mixture of characteristic functions could open new possibilities for fast data analysis based on exact probability distributions in areas like multidimensional statistics, measurement uncertainty analysis in metrology as well as in financial mathematics and risk analysis. 

* Johansson, F. (2017). Arb: Efficient Arbitrary-Precision Midpoint-Radius Interval Arithmetic. IEEE Transactions on Computers, 66(8), 1281–1292. https://doi.org/10.1109/TC.2017.2690633  
* Johansson, F. (2019). Computing Hypergeometric Functions Rigorously. ACM Transactions on Mathematical Software, 45(3), 30:1-30:26. https://doi.org/10.1145/3328732  